[section title](#intro)

# Задача 

Напишите агента, набирающего как можно больше баллов в игре MountainCar-V0

# Методы решения 

Попробуем решить задачу двумя способами и сравнить результаты:
- Сделаем выборку из большого числа "успешных" игр и обучим нейронную сеть
    - Так как мы получаем награду, отличную от $-1$ только в том случае, когда машинка пересекает отметку в $0.5$ (что при случайных действиях случается экстремально редко), изменим функцию награды так, что значение награды за итерацию увеличивается на $1$, если мы достигли отметку в $-0.3$ (это значение нам подойдёт, так как стартовая позиция принимает значения от $-0.6$ до $-0.4$).
    - Игра будет считаться успешной, если за $200$ максимально возможных итераций суммарное значение награды составило больше $-185$.
- DQN

# Решение

In [101]:
import gym
import random
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models     import Sequential
from tensorflow.keras.layers     import Dense, Dropout
from tensorflow.keras.optimizers import Adam

Сначала напишем метод, который будет сравнивать эффективность моделей. Он будет принимать модель, которая играет $100$ игр и считает среднее количество награды за игру (расчёт награды не меняется относительно условия задачи).

In [145]:
def model_test(test_model):
    
    # Список из суммарных наград за 1 игру.
    scores = [] 
    
    games_number = 100
    
    # Пробегаемся по 100 играм.
    for each_game in range(games_number):    
        
        score = 0
        
        # Состояние среды за предыдущую итерацию. 
        # Будем его передавать в качестве параметра для нашей модели.
        previous_observation = []
        
        for step_index in range(goal_steps):
            
            # Если это первая итерация, то выбераем случайное действие.
            if len(previous_observation)==0:
                action = random.randrange(0,2)
            # Иначе подаём модели прошлой состояние и выбираем действие с наибольшим весом.
            else:
                action = np.argmax(test_model.predict(previous_observation.reshape(-1, env.observation_space.shape[0]))[0])

            new_observation, reward, done, info = env.step(action)
            previous_observation = new_observation
            score += reward
            
            if done:            
                break
                
        env.reset()
        scores.append(score)
            
    print('Средняя награда за {} игр: {}'.format(games_number, sum(scores)/len(scores)))

## Решение №1 

Напишем класс, содержащий всю логику решения.

In [163]:
class NN:
    def __init__(self, env):
        self.env = env
        
        self.goal_steps = 200
        self.score_requirement = -185
        self.games_number = 10000

        self.model = self.create_model()

    # Создание модели.
    def create_model(self):
        model = Sequential([
            Dense(512, input_dim=self.env.observation_space.shape[0], activation='relu'),
            Dense(256, input_dim=input_size, activation='relu'),
            Dense(128, input_dim=input_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.env.action_space.n, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(), metrics=['acc'])
        return model
    
    # Обучение модели.
    def train_model(self, x, y):        
        self.model.fit(x, y, epochs=5)

    # Подготовка данных для обучения.
    def data_preparation(self):
        
        # Сюда будем записывать информацию об играх, которые прошли отбор по награде.
        training_data = []
        
        # Сюда будем записывать количество очков в играх, которые прошли отбор.
        accepted_scores = []
        
        # Пробегаемся по всем играм.
        for game_index in range(self.games_number):
            
            score = 0
            
            # Сюда запишем всю историю
            game_memory = []
            previous_observation = []
            
            for step_index in range(self.goal_steps):
                action = random.randrange(0, 3)
                observation, reward, done, info = env.step(action)

                if len(previous_observation) > 0:
                    game_memory.append([previous_observation, action])

                previous_observation = observation
                if observation[0] > -0.3:
                    reward = 1

                score += reward
                if done:
                    break

            # Если к концу игры мы набрали нужное количество награды, то 
            # все состояния и принятые решение записываем в training data.
            if score >= self.score_requirement:
                accepted_scores.append(score)
                for data in game_memory:
                    if data[1] == 1:
                        output = [0, 1, 0]
                    elif data[1] == 0:
                        output = [1, 0, 0]
                    elif data[1] == 2:
                        output = [0, 0, 1]
                    training_data.append([data[0], output])

            env.reset()
            
            if (game_index%1000==0):
                print('Завершено {} итераций из {}.'.format(game_index, self.games_number))

        print("Количество игр, данные по которым будут использованы для обучения: ",len(accepted_scores))
        
        # Составляем массивы для обучения (x – состояние, y – решение)
        x = np.array([i[0] for i in training_data]).reshape(-1, self.env.observation_space.shape[0])
        y = np.array([i[1] for i in training_data]).reshape(-1, self.env.action_space.n)

        return x, y

Протестируем данные метод.

In [164]:
env = gym.make("MountainCar-v0")
env.reset()

array([-0.52396908,  0.        ])

In [165]:
nn_agent = NN(env)

In [166]:
x, y = nn_agent.data_preparation()

Завершено 0 итераций из 10000.
Завершено 1000 итераций из 10000.
Завершено 2000 итераций из 10000.
Завершено 3000 итераций из 10000.
Завершено 4000 итераций из 10000.
Завершено 5000 итераций из 10000.
Завершено 6000 итераций из 10000.
Завершено 7000 итераций из 10000.
Завершено 8000 итераций из 10000.
Завершено 9000 итераций из 10000.
Количество игр, данные по которым будут использованы для обучения:  577


In [167]:
nn_agent.train_model(x, y)

Epoch 1/5
114823/114823 [==============================] - 17s 146us/sample - loss: 0.2222 - acc: 0.3550
Epoch 2/5
114823/114823 [==============================] - 16s 143us/sample - loss: 0.2211 - acc: 0.3754
Epoch 3/5
114823/114823 [==============================] - 16s 143us/sample - loss: 0.2209 - acc: 0.3771
Epoch 4/5
114823/114823 [==============================] - 16s 144us/sample - loss: 0.2207 - acc: 0.3763
Epoch 5/5
114823/114823 [==============================] - 17s 147us/sample - loss: 0.2206 - acc: 0.3766


In [168]:
model_test(nn_agent.model)

Средняя награда за 100 игр: -146.71


## Решение №2

In [2]:
class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=2000)
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.tau = .125

        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(24, input_dim=state_shape[0], activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = 32
        if len(self.memory) < batch_size: 
            return

        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

In [3]:
from collections import deque

In [4]:
env     = gym.make("MountainCar-v0")
gamma   = 0.9
epsilon = .95

trials  = 1000
trial_len = 500

# updateTargetNetwork = 1000
dqn_agent = DQN(env=env)
steps = []
for trial in range(trials):
    cur_state = env.reset().reshape(1,2)
    for step in range(trial_len):
        action = dqn_agent.act(cur_state)
        new_state, reward, done, _ = env.step(action)

        # reward = reward if not done else -20
        new_state = new_state.reshape(1,2)
        dqn_agent.remember(cur_state, action, reward, new_state, done)

        dqn_agent.replay()       # internally iterates default (prediction) model
        dqn_agent.target_train() # iterates target model

        cur_state = new_state
        if done:
            break
    if step >= 199:
        print("Failed to complete in trial {}".format(trial))
        if step % 10 == 0:
            dqn_agent.save_model("trial-{}.model".format(trial))
    else:
        print("Completed in {} trials".format(trial))
        dqn_agent.save_model("success.model")
        break


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Failed to complete in trial 0
Failed to complete in trial 1
Failed to complete in trial 2
Failed to complete in trial 3
Failed to complete in trial 4
Failed to complete in trial 5
Failed to complete in trial 6
Failed to complete in trial 7
Failed to complete in trial 8
Failed to complete in trial 9
Failed to complete in trial 10
Failed to complete in trial 11


KeyboardInterrupt: 